In [1]:
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path

In [67]:
import time

In [7]:
txt = Path('data/aapl/2016-Apr-26-AAPL.txt').read_text()

In [9]:
splitter = "================================================================================"
txt.split(splitter)

['\n\nThomson Reuters StreetEvents Event Transcript\nE D I T E D   V E R S I O N\n\nQ2 2016 Apple Inc Earnings Call\nAPRIL 26, 2016 / 9:00PM GMT\n\n',
 '\nCorporate Participants\n',
 '\n\n * Luca Maestri\n   Apple Inc. - CFO\n * Tim Cook\n   Apple Inc. - CEO\n * Nancy Paxton\n   Apple Inc. - Senior Director of IR\n\n',
 '\nConference Call Participiants\n',
 '\n\n * Katy Huberty\n   Morgan Stanley - Analyst\n * Gene Munster\n   Piper Jaffray & Co. - Analyst\n * Rod Hall\n   JPMorgan - Analyst\n * Shannon Cross\n   Cross Research - Analyst\n * Toni Sacconaghi\n   Bernstein - Analyst\n * Simona Jankowski\n   Goldman Sachs - Analyst\n * Steve Milunovich\n   UBS - Analyst\n\n',
 "\nPresentation\n--------------------------------------------------------------------------------\nOperator    [1]\n--------------------------------------------------------------------------------\n\n          Good day, everyone, and welcome to the Apple, Incorporated second-quarter FY16 earnings release conference 

In [10]:
splitter = "--------------------------------------------------------------------------------"
txt.split(splitter)

['\n\nThomson Reuters StreetEvents Event Transcript\nE D I T E D   V E R S I O N\n\nQ2 2016 Apple Inc Earnings Call\nAPRIL 26, 2016 / 9:00PM GMT\n\n================================================================================\nCorporate Participants\n================================================================================\n\n * Luca Maestri\n   Apple Inc. - CFO\n * Tim Cook\n   Apple Inc. - CEO\n * Nancy Paxton\n   Apple Inc. - Senior Director of IR\n\n================================================================================\nConference Call Participiants\n================================================================================\n\n * Katy Huberty\n   Morgan Stanley - Analyst\n * Gene Munster\n   Piper Jaffray & Co. - Analyst\n * Rod Hall\n   JPMorgan - Analyst\n * Shannon Cross\n   Cross Research - Analyst\n * Toni Sacconaghi\n   Bernstein - Analyst\n * Simona Jankowski\n   Goldman Sachs - Analyst\n * Steve Milunovich\n   UBS - Analyst\n\n=====================

In [16]:
!pip install openai

     ---------------------------------------- 72.0/72.0 kB 1.3 MB/s eta 0:00:00


In [18]:
import os
import openai

In [28]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [22]:
passages_df = pd.read_csv('passages.csv')

In [48]:
len(passages_df)

7167

In [23]:
passages_df.iloc[5]['passage']

' Edited version Q2 2016 Apple Inc Earnings Call: First, from an upgrade perspective, during the first half of this year, the upgrade rate for the iPhone 6s cycle has been slightly higher than what we experienced in the iPhone 5s cycle two years ago, but it is lower than the accelerated upgrade rate we saw with iPhone 6, which as you know, was a big contributor to our phenomenal revenue growth a year ago. Most importantly, our customers are incredibly loyal. A recent Kantar survey of US smart phone purchasers indicated a 95% iPhone loyalty rate, the highest ever measured for any smart phone. \nSecond, we continued to see a very high level of customers switching to iPhone from Android and other operating systems. In fact, we added more switchers from Android and other platforms in the first half of this year than any other six-month period ever.'

In [30]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=passages_df.iloc[5]['passage'] + " Generate question and answer from above paragraph",
  temperature=0.9,
  max_tokens=250,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0.6,
  stop=[" Human:", " AI:"]
)

AuthenticationError: <empty message>

In [65]:
import os
import openai

openai.api_key = ""

response = openai.Completion.create(
  model="text-curie-001",
  prompt=passages_df.iloc[15]['passage'] + " Generate one question and an answer from above paragraph",
  temperature=0.1,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0.6
)

In [66]:
response["choices"][0]['text']

'.\n\nQ: What acquisitions have been made in the last four quarters?\nA: In the last four quarters, Apple has acquired 15 companies, including Beats Music, Autodesk, and Texture.'

In [43]:
passages_df.iloc[6]['passage']

" Edited version Q2 2016 Apple Inc Earnings Call: \nAnd third, with only 42% smart phone penetration of the global handset market today, iPhone is still attracting millions of first-time smart phone buyers each quarter, especially from emerging markets. For example, in India, our iPhone sales were up 56% from a year ago. \nNext, I'd like to talk about services, which was our second-largest revenue generating category during the quarter. Setting aside the amount we received from a patent settlement in the December quarter, the March quarter services revenue was our highest ever. Services revenue jumped 20% to $6 billion. App Store revenue was up 35% to beat last quarter's all-time record, and Apple Music continues to grow in popularity with over 13 million paying subscribers today."

In [73]:
def get_gpt_response(passage):
    response = openai.Completion.create(
      model="text-curie-001",
      prompt=passage + " Generate one question and an answer from above paragraph",
      temperature=0.1,
      max_tokens=512,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0.6
    )
    return response["choices"][0]['text']

In [74]:
import time

In [77]:
qna_passage = []
for index in range(0, len(passages_df.head(1000))):
    qna = get_gpt_response(passages_df.iloc[index]['passage'])
    document = passages_df.iloc[index]['document']
    passage = passages_df.iloc[index]['passage']
    qna_passage.append([document, passage, qna])
    
    time.sleep(0.01)
    
    if index%50 == 0:
        pd.DataFrame(qna_passage).to_csv('qna_passage.csv', index=False)
        print(index)
        time.sleep(90)
    

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [79]:
for index in range(1000, 2000):
    if index%50 == 0:
        print(index)


1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [82]:
passages_df.iloc[7050]['document']

'Edited version Q2 2019 NVIDIA Corp Earnings Call'

In [83]:
qna_passage = []
for index in range(1000, 2000):
    qna = get_gpt_response(passages_df.iloc[index]['passage'])
    document = passages_df.iloc[index]['document']
    passage = passages_df.iloc[index]['passage']
    qna_passage.append([document, passage, qna])
    
    time.sleep(0.01)
    
    if index%50 == 0:
        pd.DataFrame(qna_passage).to_csv('qna_passage.csv', index=False)
        print(index)
        time.sleep(60)
    

1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [86]:
qna_passage = []
for index in range(3001, 4001):
    qna = get_gpt_response(passages_df.iloc[index]['passage'])
    document = passages_df.iloc[index]['document']
    passage = passages_df.iloc[index]['passage']
    qna_passage.append([document, passage, qna])
    
    time.sleep(0.01)
    
    if index%50 == 0:
        pd.DataFrame(qna_passage).to_csv('qna_passage_4000.csv', index=False)
        print(index)
        time.sleep(60)
    

3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000


In [87]:
qna_passage = []
for index in range(4001, 5001):
    qna = get_gpt_response(passages_df.iloc[index]['passage'])
    document = passages_df.iloc[index]['document']
    passage = passages_df.iloc[index]['passage']
    qna_passage.append([document, passage, qna])
    
    time.sleep(0.01)
    
    if index%50 == 0:
        pd.DataFrame(qna_passage).to_csv('qna_passage_5000.csv', index=False)
        print(index)
        time.sleep(50)
    

4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
